## Author: Ben Cliff

# About the data  
##### The dataset loaded into this notebook is a .csv file obtained from the CDC's preventative measures site.  
##### The sample subject of this dataset is U.S. citizens who are 65 years or older. The sample is subdivided by gender, State and city.
##### The target or signal of this dataset is the percentage of citizens (broken down by gender and location) who are reported to have taken preventative measures such as immunizations and select cancer screenings against common illnesses.
##### [Bethlehem insert commentary here about what trends we want to see]  

# Purpose of this notebook:
##### Create a clean .csv file to work from for further analysis
##### Perform preliminary analysis as well as diagnostics of the data
##### Create visualizations from this notebook for upload onto our blog site

# Reading and cleaning the dataset

In [1]:
# Importing powerful data manipulation library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
# import shapely
from shapely.geometry import Point

In [2]:
# Reading in prevention data
prevent_df = pd.read_csv('../data/preventativedata.csv')

# Removing with little to no information
prevent_df = prevent_df.drop(columns=['Data_Value_Unit', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'TractFIPS', 'CategoryID', 'StateDesc', 'Data_Value_Type', 'DataSource',
'DataValueTypeID', 'Category'])

In [3]:
# Removing undescores
prevent_df.columns = prevent_df.columns.str.replace('_', '')
# prevent_df.columns

In [4]:
# Sorting values by unique ID; creating new index and dropping the old one
prevent_df.sort_values(by=['UniqueID'], ascending=True).reset_index().head(3).drop(columns=['index'])

,Year,StateAbbr,CityName,GeographicLevel,UniqueID,Measure,DataValue,LowConfidenceLimit,HighConfidenceLimit,PopulationCount,GeoLocation,MeasureId,CityFIPS,ShortQuestionText
0,2016,HI,Honolulu,City,15003,Older adult men aged >=65 Years who are up to ...,31.4,31.1,31.7,"953,207","(21.4588039305, -157.973296737)",COREM,15003,Core preventive services for older men
1,2016,HI,Honolulu,City,15003,Older adult women aged >=65 Years who are up t...,30.4,30.1,30.7,"953,207","(21.4588039305, -157.973296737)",COREW,15003,Core preventive services for older women
2,2016,AL,Birmingham,City,107000,Older adult men aged >=65 Years who are up to ...,32.9,32.2,33.7,"212,237","(33.5275663773, -86.7988174678)",COREM,107000,Core preventive services for older men


In [5]:
temp_list = []

longs = []

prevent_df['GeoLocation'] = [x.replace('(', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.replace(')', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.split(',') for x in prevent_df['GeoLocation']]
#prevent_df['GeoLocation'] = [float(x) for x in prevent_df['GeoLocation']]


In [6]:
lats = [float(x[0]) for x in prevent_df['GeoLocation']]
longs = [float(x[1]) for x in prevent_df['GeoLocation']]

In [7]:
prevent_df['lats'] = lats
prevent_df['longs'] = longs
# prevent_df.head(5)

# Data Visualization and Analysis

In [8]:
# Importing visualization tool
import plotly.graph_objects as go
import plotly.express as px

In [9]:
prevent_df['PopulationCount'] = [x.replace(',', '') for x in prevent_df['PopulationCount']]
prevent_df['PopulationCount'] = prevent_df['PopulationCount'].astype(int)

In [10]:
# Adding new column to prevent_df: top 5 states for geospatial analysis
temp_list = []
for x in prevent_df['StateAbbr']:
    if x == 'CA' or x == 'TX' or x == 'FL' or x == 'IL' or x == 'MI':
        temp_list.append('yes')
    else:
        temp_list.append('no')

In [11]:
prevent_df['top5state'] = temp_list
# prevent_df.head(10)

In [12]:
# Extracting only the male records of the dataset to get the unique counts of surveys by cities
men_df = prevent_df.loc[prevent_df['MeasureId'] == 'COREM']

# Isolating the top 5 counts of cities in separate dataframe
top_5_states = men_df.loc[(men_df['StateAbbr'] == 'CA') | (men_df['StateAbbr'] == 'TX') | (men_df['StateAbbr'] == 'FL')
| (men_df['StateAbbr'] == 'IL') | (men_df['StateAbbr'] == 'MI')]

# Isolating all other 45 states in separate dataframe
other_states_df = men_df.loc[(men_df['StateAbbr'] != 'CA') & (men_df['StateAbbr'] != 'TX') & (men_df['StateAbbr'] != 'FL')
& (men_df['StateAbbr'] != 'IL') & (men_df['StateAbbr'] != 'MI')]

In [13]:
# Creating separate data for the histogram counts
x = other_states_df['StateAbbr']
y = top_5_states['StateAbbr']

# Creating graph object
fig = go.Figure()
fig.add_trace(go.Histogram(histfunc='count', x=x, name='State Participants'))
fig.add_trace(go.Histogram(histfunc='count', x=y, name='Top 5 States represented', marker_color='#330C73'))
fig.update_layout(title_text='Cities Count of Survey', xaxis_title_text='State Abbreviation', yaxis_title_text='Count')
fig.show()

In [21]:

geometry = [Point(xy) for xy in zip(lats, longs)]
gdf = GeoDataFrame(prevent_df, geometry=geometry)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world.loc[world['name'] == 'United States of America']

fig = px.scatter_geo(men_df, lat=men_df['lats'], lon=men_df['longs'], width=800, height=400, color=men_df['top5state'])
fig.update_layout(title = 'PLACES survey locations', geo_scope='usa', showlegend=False)
fig.show()

In [15]:

# geometry = [Point(xy) for xy in zip(lats, longs)]
# gdf = GeoDataFrame(prevent_df, geometry=geometry)

# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# usa = world.loc[world['name'] == 'United States of America']

# fig = px.scatter_geo(prevent_df, lat=top_5_states['lats'], lon=top_5_states['longs'], width =800, height=400)
# fig.update_traces(marker=dict(color='red'))
# fig.update_layout(title = 'PLACES survey locations of the top 5 most surveyed states', geo_scope='usa')
# fig.show()

In [16]:
prevent_df.describe()

,Year,UniqueID,DataValue,LowConfidenceLimit,HighConfidenceLimit,PopulationCount,CityFIPS,lats,longs
count,1000.0,1.000000e+03,1000.000000,1000.00000,1000.000000,1.000000e+03,1.000000e+03,1000.000000,1000.000000
mean,2016.0,2.438986e+06,32.559500,31.46880,33.654300,2.060416e+05,2.438986e+06,36.970325,-97.988528
std,0.0,1.719433e+06,5.522466,5.37843,5.684762,4.572520e+05,1.719433e+06,5.256474,17.470177
min,2016.0,1.500300e+04,17.300000,16.30000,17.900000,4.241700e+04,1.500300e+04,21.458804,-157.973297
25%,2016.0,6.700735e+05,29.100000,28.10000,29.900000,8.159000e+04,6.700735e+05,33.675199,-117.390113
50%,2016.0,1.955965e+06,32.450000,31.30000,33.500000,1.061060e+05,1.955965e+06,37.149690,-95.312924
75%,2016.0,4.053125e+06,36.100000,34.90000,37.300000,1.811278e+05,4.053125e+06,41.100725,-82.732076
max,2016.0,5.613900e+06,53.000000,50.70000,55.300000,8.175133e+06,5.613900e+06,61.149869,-70.275254


In [24]:
# Cross tabulation of the Data by State and the Population Count of each survey location
new_df = pd.crosstab(index=[men_df['StateAbbr'], pd.cut(men_df['PopulationCount'], [0, 50000, 100000, 250000, 500000, 1000000])], columns=men_df['PopulationCount'], margins=True, margins_name='Counts')

In [25]:
# Find a state's location site's population by state abbreviation
new_df.loc['TX'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,3


In [26]:
new_df.loc['CA'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,55
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,53
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,2


In [27]:
new_df.loc['FL'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
